## LangModel-RNN - Implementing a language model recurrent neural network

The goal here is to build a [Language Model](https://en.wikipedia.org/wiki/Language_model) from scratch using a Recurrent Neural Network. This work is heavily inspired by the [awesome article](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/) at wildml.com explaining RNNs.

For a clear understanding of RNNs, I suggest one should check out Christopher Olah's [blog](http://colah.github.io/posts/2015-08-Understanding-LSTMs/). Also, there is a [fun post](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) by Karpathy demonstrating what character level language models are capable of. Do check them out once you are done here.

In [ ]:
import csv
import itertools
import operator
import sys
import nltk
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from utils import *

%matplotlib inline

In [ ]:
# We need to download the NLTK model data (this step needs to be performed once)
# nltk.download("book")

# Edit if you have downloaded nltk_data in a different path
nltk.data.path.append('/home/saikat/.local/share/libs/nltk_data/')

### Training Data and Preprocessing

The training data can be simple text. We don't need any labels to train a language model. I'm using a collection of Reddit comments. Our trained model thus should be able to generate texts that sound like Reddit comments.

And, obviously, we cannot just feed text as input to our network. We need to do some pre-processing.

1. Tokenize Text

We want to do prediction on a per word basis. That is, given a word the network should be able to predict the next word. To achieve this, we need to break our comments into sentences, and sentences into words. For example, "that is wonderful!" should be 4 tokens, "that", "is", "wonderful" and "!". We'll use NLTK for the job. NLTK will help us manage punctuations properly.

2. Remove infrequent words

Removing words that come infrequently will help us keep a relatively smaller vocabulary. Having a huge vocabulary will make our model slow to train, and because we don't have a lot of contextual examples for such words we wouldn't be able to learn how to use them correctly anyway. This approach is quite similar to how humans learn and this is pretty effective in practice.

We'll keep the ``vocabulary_size`` to some value that is reasonable (I have a dell laptop with a 2nd gen i5). We'll replace any word that does not come in the vocabulary with the ``UNKNOWN_TOKEN``.

3. Prepend special start and end tokens

We'll use special start and end tokens ``SENTENCE_START`` and ``SENTENCE_END`` so signify start and end of a sentence.

4. Building the training data matrices

The input to our RNNs are vectors, not strings. So we create a mapping between words and indices. For example, the word "friendly" may be at index 2001. A training example ￼ may look like [0, 179, 341, 416], where 0 corresponds to ``SENTENCE_START``. The corresponding label ￼ would be [179, 341, 416, 1]. Remember that our goal is to predict the next word, so ``y`` is just the ``x`` vector shifted by one position to the left with the last element being the ``SENTENCE_END`` token. In other words, the correct prediction for word 179 above would be 341, the actual next word.

In [ ]:
vocabulary_size      = 1000 # Keeping this small for the time being
unknown_token        = "UNKNOWN_TOKEN"
sentence_start_token = "SENTENCE_START"
sentence_end_token   = "SENTENCE_END"

# Read the data and append SENTENCE_START and SENTENCE_END tokens
print "Reading CSV file..."
with open('data/reddit-comments-2015-08.csv', 'rb') as f:
    reader = csv.reader(f, skipinitialspace=True)
    reader.next()
    # Split full comments into sentences
    sentences = itertools.chain(
                    *[nltk.sent_tokenize(x[0].decode('utf-8').lower()) for x in reader]
                )
    # Append SENTENCE_START and SENTENCE_END
    sentences = ["%s %s %s" % (sentence_start_token, x, sentence_end_token) for x in sentences]
print "Parsed %d sentences." % (len(sentences))

# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(x) for x in sentences]